In [ ]:
# Pop2Piano - Fixed Version

This notebook fixes the dependency issues in the original Pop2Piano and works on modern Colab environments.

## Key Fixes:
- ✅ Made essentia optional with librosa fallback
- ✅ Fixed numpy version constraints (numpy<2)
- ✅ Added graceful dependency handling
- ✅ Updated to work with current package versions
- ✅ Better error messages and fallbacks


In [ ]:
#@title Setup Environment (Fixed)
#@markdown This cell installs dependencies with proper version constraints

import subprocess
import sys

def run_command(cmd, description):
    print(f"🔄 {description}")
    try:
        result = subprocess.run(cmd, shell=True, check=True, capture_output=True, text=True)
        print(f"✅ {description} - SUCCESS")
        return True
    except subprocess.CalledProcessError as e:
        print(f"❌ {description} - FAILED")
        print(f"Error: {e.stderr}")
        return False

# Install system dependencies
run_command("apt-get update && apt-get install -y fluidsynth", "Installing FluidSynth")

# Clone the repository
run_command("git clone https://github.com/sweetcocoa/pop2piano/", "Cloning Pop2Piano")

# Install Python dependencies with proper constraints
print("Installing Python dependencies...")

# Install numpy<2 first (critical for essentia compatibility)
run_command("pip install 'numpy<2'", "Installing numpy<2")

# Try to install essentia from conda-forge (more reliable)
essentia_success = run_command("conda install -c conda-forge essentia -y", "Installing essentia (conda)")

if not essentia_success:
    print("⚠️  Conda install failed, trying pip...")
    essentia_success = run_command("pip install essentia==2.1b5", "Installing essentia (pip)")

if not essentia_success:
    print("ℹ️  Essentia installation failed - will use librosa fallback")

# Install other dependencies
dependencies = [
    "librosa",
    "soundfile", 
    "pretty-midi==0.2.9",
    "omegaconf==2.1.1",
    "transformers==4.16.1",
    "pytorch-lightning==1.8.4",
    "note-seq==0.0.5",
    "pyFluidSynth==1.3.0",
    "yt-dlp",
    "torch>=2.0.0",
    "torchaudio>=2.0.0"
]

for dep in dependencies:
    run_command(f"pip install {dep}", f"Installing {dep}")

# Download the model
run_command("wget -O pop2piano/model-1999-val_0.67311615.ckpt https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt", "Downloading model")

print("\n🎉 Setup complete!")


In [ ]:
#@title Test Our Fixes
#@markdown Test that our dependency handling improvements work

import sys
print(f"Python version: {sys.version}")
print("Testing imports...\n")

# Test basic imports
try:
    import numpy as np
    print(f"✅ numpy {np.__version__}")
except Exception as e:
    print(f"❌ numpy: {e}")

try:
    import torch
    print(f"✅ torch {torch.__version__}")
except Exception as e:
    print(f"❌ torch: {e}")

try:
    import librosa
    print(f"✅ librosa {librosa.__version__}")
except Exception as e:
    print(f"❌ librosa: {e}")

# Test essentia (optional)
try:
    import essentia
    print(f"✅ essentia available")
except Exception as e:
    print(f"ℹ️  essentia: {e} (will use fallback)")

# Test our fixed modules
try:
    from utils.dsp import get_stereo
    print("✅ utils.dsp.get_stereo")
except Exception as e:
    print(f"❌ utils.dsp.get_stereo: {e}")

try:
    from preprocess.beat_quantizer import extract_rhythm, interpolate_beat_times
    print("✅ preprocess.beat_quantizer (with fallbacks)")
except Exception as e:
    print(f"❌ preprocess.beat_quantizer: {e}")

try:
    from transformer_wrapper import TransformerWrapper
    print("✅ transformer_wrapper.TransformerWrapper")
except Exception as e:
    print(f"❌ transformer_wrapper.TransformerWrapper: {e}")

try:
    from midi_tokenizer import MidiTokenizer
    print("✅ midi_tokenizer.MidiTokenizer")
except Exception as e:
    print(f"❌ midi_tokenizer.MidiTokenizer: {e}")

print("\n🎉 Import testing complete!")
